## 🧠 Copilot Prompt

Generate a GitHub Actions workflow that automatically runs this notebook using `jupyter nbconvert --execute` and exports the HTML to `docs/validation.html`.

Ensure that `mpmath`, `sympy`, and `numpy` are installed beforehand.

If execution fails, output a warning with traceback.

In [ ]:
import mpmath as mp
import sympy as sp
import os

# Conditional precision and parameters based on environment
# High precision for research, lower for CI to avoid timeouts
is_ci = os.getenv('CI', 'false').lower() == 'true'
if is_ci:
    mp.mp.dps = 15  # Reduced precision for CI
    DEFAULT_P = 100   # Fewer primes
    DEFAULT_K = 10    # Fewer powers
    DEFAULT_N = 50    # Fewer zeros
    DEFAULT_T = 10    # Smaller integration range
    print("🤖 Running in CI mode with reduced parameters for faster execution")
else:
    mp.mp.dps = 50   # Full precision for research
    DEFAULT_P = 1000
    DEFAULT_K = 50
    DEFAULT_N = 2000
    DEFAULT_T = 50
    print("🔬 Running in research mode with full precision")

def f1(u): return mp.exp(-u**2/2) if abs(u) <= 3 else mp.mpf(0)
def f2(u): return mp.exp(-u**2) if abs(u) <= 2 else mp.mpf(0)
def f3(u): return (1 - u**2/4)**2 if abs(u) <= 2 else mp.mpf(0)

def fhat(f, s, lim):
    # Reduce integration precision for CI
    maxdegree = 5 if is_ci else 10
    return mp.quad(lambda u: f(u) * mp.exp(s * u), [-lim, lim], maxdegree=maxdegree)

def prime_sum(f, P=None, K=None):
    if P is None: P = DEFAULT_P
    if K is None: K = DEFAULT_K
    s = mp.mpf(0)
    primes = list(sp.primerange(2, 7919))[:P]
    for p in primes:
        lp = mp.log(p)
        for k in range(1, K+1):
            s += lp * f(k * lp)
    return s

def A_infty(f, sigma0=2.0, lim=3, T=None):
    if T is None: T = DEFAULT_T
    def integrand(t):
        s = mp.mpc(sigma0, t)
        return (mp.digamma(s/2) - mp.log(mp.pi)) * fhat(f, s, lim)
    # Reduce integration precision for CI
    maxdegree = 5 if is_ci else 10
    integ = mp.quad(integrand, [-T, T], maxdegree=maxdegree) / (2 * mp.pi)
    res1 = fhat(f, mp.mpf(1), lim) / mp.mpf(1)
    return integ - res1

def zero_sum(f, N=None, lim=3):
    if N is None: N = DEFAULT_N
    s = mp.mpf(0)
    for n in range(1, N+1):
        rho = mp.zetazero(n)
        s += fhat(f, mp.im(rho), lim).real
    return s

print(f"\n📊 Validation Parameters:")
print(f"   Precision: {mp.mp.dps} digits")
print(f"   Primes: {DEFAULT_P}, Powers: {DEFAULT_K}")
print(f"   Zeros: {DEFAULT_N}, Integration range: ±{DEFAULT_T}")
print("\n🧮 Computing explicit formula validation...\n")

for fname, f, lim in [('f1', f1, 3), ('f2', f2, 2), ('f3', f3, 2)]:
    print(f"Computing {fname}...")
    ps = prime_sum(f)
    ain = A_infty(f, lim=lim)
    zs = zero_sum(f, lim=lim)
    tot = ps + ain
    err = abs(tot - zs)
    rel_err = err / abs(tot) if abs(tot) > 0 else 0
    print(f'✅ {fname}: prime+A={tot:.6f}, zero={zs:.6f}, err={err:.2e}, rel_err={rel_err:.2e}\n')

print("🎉 Validation completed successfully!")